# Prototyping an ML Model
## Prerequisites

In [1]:
import glob
import pandas as pd
from mmproteo.utils.utils import ensure_dir_exists
from mmproteo.utils import log
from mmproteo.utils.formats.mz import FilteringProcessor, filter_files
from mmproteo.utils.processing import ItemProcessor
import os
import tensorflow as tf
import numpy as np
from typing import Iterable, Callable, Dict, Any, Tuple
import gc

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [3]:
logger = log.DummyLogger(verbose=False)

INFO: Printing to Stdout


## Configuration

In [4]:
pwd

'/tf/workspace/notebooks'

In [5]:
PROJECT = "PXD010000"
DUMP_PATH = os.path.join("..", "dumps", PROJECT)
TRAINING_COLUMNS_DUMP_PATH = os.path.join(DUMP_PATH, "training_columns")
FILES_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "*_mzmlid.parquet")
DATASET_DUMP_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "tf_datasets")

In [6]:
MZMLID_FILE_PATHS = glob.glob(FILES_PATH)
len(MZMLID_FILE_PATHS)

235

In [7]:
MZMLID_FILE_PATHS[0]

'../dumps/PXD010000/training_columns/Biodiversity_S_agalactiae_LIB_aerobic_02_26Feb16_Arwen_16-01-01_mzmlid.parquet'

In [8]:
SEQ = FilteringProcessor.default_peptide_sequence_column_name
MZ = FilteringProcessor.default_mz_array_column_name
INT = FilteringProcessor.default_intensity_array_column_name

## Calculating Statistics over all MZMLID Files

In [9]:
STATISTICS_FILE_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "statistics.parquet")

In [10]:
file_path_count = len(MZMLID_FILE_PATHS)

def get_mzmlid_file_stats(item: Tuple[int, str]) -> Dict[str, Any]:
    idx, path = item
    info_text = f"Processing item {idx + 1}/{file_path_count} '{path}'"
    if idx % 10 == 0:
        logger.info(info_text)
    else:
        logger.debug(info_text)
    df = pd.read_parquet(path)
    max_sequence_length = df[SEQ].str.replace(r"[^A-Z]",'').str.len().max()
    max_array_length = df[INT].str.len().max()
    alphabet = set.union(*df[SEQ].apply(set))
    item_count = len(df)
    del df
    gc.collect()
    
    return {
        "file_path": path,
        "max_sequence_length": max_sequence_length,
        "max_array_length": max_array_length,
        "alphabet": alphabet,
        "item_count": item_count
    }

if os.path.exists(STATISTICS_FILE_PATH):
    file_stats = pd.read_parquet(STATISTICS_FILE_PATH)
    file_stats.alphabet = file_stats.alphabet.apply(set)
    print("loaded previous statistics")
else:
    file_stats = pd.DataFrame(
        ItemProcessor(
            items=enumerate(MZMLID_FILE_PATHS),
            item_processor=get_mzmlid_file_stats,
            action_name="analyse",
            subject_name="mzmlid file",
            thread_count=0,
            logger=logger
        ).process()
    )
    
    file_stats_writable = file_stats.copy()
    file_stats_writable.alphabet = file_stats_writable.alphabet.apply(list) # cannot store sets
    file_stats_writable.write_parquet(STATISTICS_FILE_PATH)

loaded previous statistics


In [11]:
file_stats.head(2)

,file_path,max_sequence_length,max_array_length,alphabet,item_count
0,../dumps/PXD010000/training_columns/Biodiversity_S_agalactiae_LIB_aerobic_02_26Feb16_Arwen_16-01-01_mzmlid.parquet,50,1267,"{A, Q, D, Y, I, T, P, L, M, F, C, N, S, G, V, W, K, R, E, H}",13279
1,../dumps/PXD010000/training_columns/Biodiversity_B_fragilis_01_28Jul15_Arwen_14-12-03_mzmlid.parquet,50,1845,"{A, Q, D, Y, T, P, E, L, M, F, C, N, S, G, V, W, K, R, I, H}",26830


In [12]:
MAX_SEQUENCE_LENGTH = file_stats.max_sequence_length.max()
print(f"MAX_SEQUENCE_LENGTH = {MAX_SEQUENCE_LENGTH}")

MAX_ARRAY_LENGTH = file_stats.max_array_length.max()
print(f"MAX_ARRAY_LENGTH = {MAX_ARRAY_LENGTH}")

TOTAL_ITEM_COUNT = file_stats.item_count.sum()
print(f"TOTAL_ITEM_COUNT = {TOTAL_ITEM_COUNT}")

ALPHABET = set.union(*file_stats.alphabet)
print(f"ALPHABET = {', '.join(sorted(ALPHABET))}")

MAX_SEQUENCE_LENGTH = 50
MAX_ARRAY_LENGTH = 2354
TOTAL_ITEM_COUNT = 5408046
ALPHABET = A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y


## Data Normalization, Padding, and Conversion to Tensorflow Datasets

In [13]:
def l2_normalize(values: np.ndarray) -> np.ndarray:
    return tf.keras.utils.normalize(x=values, order=2)

In [14]:
def base_peak_normalize(values: np.ndarray) -> np.ndarray:
    return values / values.max()

In [15]:
# by Tom, probably
# don't know, what it's based on
def ion_current_normalize(intensities):
    total_sum = np.sum(intensities**2)
    normalized = intensities/total_sum
    return normalized

In [16]:
NORMALIZATION=base_peak_normalize

In [17]:
PADDING_CHARACTERS = {
    SEQ: '_',
    MZ: 0.0,
    INT: 0.0,
}

In [18]:
ALPHABET.add(PADDING_CHARACTERS[SEQ])

In [19]:
char_to_idx = {char: idx for idx, char in enumerate(sorted(ALPHABET))}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}
INDEX_ALPHABET = idx_to_char.keys()
char_to_idx

{'A': 0,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'K': 8,
 'L': 9,
 'M': 10,
 'N': 11,
 'P': 12,
 'Q': 13,
 'R': 14,
 'S': 15,
 'T': 16,
 'V': 17,
 'W': 18,
 'Y': 19,
 '_': 20}

In [20]:
ARRAY_COLS = [MZ, INT]

In [21]:
def normalize_intensities(df: pd.DataFrame):
    df[INT] = df[INT].apply(NORMALIZATION)

def pad_sequence_column(df: pd.DataFrame):
    df[SEQ] = df[SEQ].str.pad(
        width=MAX_SEQUENCE_LENGTH, 
        fillchar=PADDING_CHARACTERS[SEQ], 
        side='right'
    )

def pad_array_columns(df: pd.DataFrame):
    for col in ARRAY_COLS:
        if len(df[col]) == 0:
            continue
        item_dtype = df[col].iloc[0].dtype

        df[col] = list(tf.keras.preprocessing.sequence.pad_sequences(
            sequences=df[col], 
            maxlen=MAX_ARRAY_LENGTH, 
            padding='post', 
            value=PADDING_CHARACTERS[col],
            dtype=item_dtype
        ))

def _sequence_to_indices(sequence: Iterable[str], 
                char_to_idx_mapping_fun: Callable[[str], int] = char_to_idx.get) -> np.ndarray:
    return np.array([char_to_idx_mapping_fun(char) for char in sequence], dtype=np.int8)

def sequence_column_to_indices(df: pd.DataFrame):
    df[SEQ] = df[SEQ].apply(list).apply(_sequence_to_indices)

def stack_numpy_arrays_in_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    return df.apply(lambda item: [np.stack(item)])

def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    normalize_intensities(df)
    pad_sequence_column(df)
    pad_array_columns(df)
    sequence_column_to_indices(df)
    return stack_numpy_arrays_in_dataframe(df)

def df2dataset(stacked_df: pd.DataFrame) -> tf.data.Dataset:
    training_data = tuple(stacked_df[ARRAY_COLS].iloc[0])
    target_data = tuple(stacked_df[[SEQ]].iloc[0])
    dataset = tf.data.Dataset.from_tensor_slices((training_data, target_data))
    return dataset
    
def parquet_file_to_dataset_file_converter(item: Tuple[int, str]) -> str:
    idx, path = item
    tf_dataset_path = os.path.join(
        DATASET_DUMP_PATH, 
        path[len(TRAINING_COLUMNS_DUMP_PATH)+len(os.path.sep):])
    if os.path.exists(tf_dataset_path):
        logger.debug(f"Skipped '{path}' because '{tf_dataset_path}' already exists")
        return None
    
    info_text = f"Processing item {idx + 1}/{len(MZMLID_FILE_PATHS)}: '{path}'"
    if idx % 10 == 0:
        logger.info(info_text)
    else:
        logger.debug(info_text)
    df = pd.read_parquet(path)
    df = preprocess_dataframe(df)
    dataset = df2dataset(df)
    logger.debug(dataset.element_spec)
    
    tf.data.experimental.save(dataset=dataset, path=tf_dataset_path, compression='GZIP')
    
    del dataset
    del df
    gc.collect()
    
    return tf_dataset_path

In [22]:
dataset_file_paths = list(ItemProcessor(
    items=enumerate(MZMLID_FILE_PATHS),
    item_processor=parquet_file_to_dataset_file_converter,
    action_name="parquet2tf_dataset-process",
    subject_name="mzmlid parquet file",
    thread_count=2,
    logger=logger
).process())
dataset_file_paths[:3]

INFO: No mzmlid parquet files were parquet2tf_dataset-processed


[]

## Loading Tensorflow Datasets

In [23]:
dataset_file_paths = glob.glob(os.path.join(DATASET_DUMP_PATH, '*'))
dataset_file_paths[:3]

['../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_HL48_HLHxylose_aerobic_2_09Jun16_Pippin_16-03-39_mzmlid.parquet',
 '../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_P_ruminicola_MDM_anaerobic_1_09Jun16_Pippin_16-03-39_mzmlid.parquet',
 '../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_C_Baltica_T240_R3_C_27Jan16_Arwen_15-07-13_mzmlid.parquet']

In [24]:
element_spec = ((tf.TensorSpec(shape=(MAX_ARRAY_LENGTH,), dtype=tf.float32), 
  tf.TensorSpec(shape=(MAX_ARRAY_LENGTH,), dtype=tf.float32)),
(tf.TensorSpec(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int8)))
element_spec

((TensorSpec(shape=(2354,), dtype=tf.float32, name=None),
  TensorSpec(shape=(2354,), dtype=tf.float32, name=None)),
 TensorSpec(shape=(50,), dtype=tf.int8, name=None))

In [25]:
datasets = [tf.data.experimental.load(path=path, element_spec=element_spec, compression='GZIP') 
            for path in dataset_file_paths]

In [26]:
datasets[:3]

[<_LoadDataset shapes: (((2354,), (2354,)), (50,)), types: ((tf.float32, tf.float32), tf.int8)>,
 <_LoadDataset shapes: (((2354,), (2354,)), (50,)), types: ((tf.float32, tf.float32), tf.int8)>,
 <_LoadDataset shapes: (((2354,), (2354,)), (50,)), types: ((tf.float32, tf.float32), tf.int8)>]

## Concatenating Tensorflow Datasets

In [27]:
BATCH_SIZE = 256

dataset = datasets[0]
for ds in datasets[1:]:
    dataset = dataset.concatenate(ds)

dataset = dataset.batch(BATCH_SIZE)

## Building the Tensorflow Model

In [28]:
input_layers = {col: tf.keras.layers.Input(shape=(MAX_ARRAY_LENGTH,)) for col in ARRAY_COLS}
input_layers

{'mz_array': <KerasTensor: shape=(None, 2354) dtype=float32 (created by layer 'input_1')>,
 'intensity_array': <KerasTensor: shape=(None, 2354) dtype=float32 (created by layer 'input_2')>}

In [29]:
x = input_layers[MZ] + input_layers[INT]

x = tf.keras.layers.Flatten()(x)

for _ in range(1):
    x = tf.keras.layers.Dense(16*MAX_SEQUENCE_LENGTH*len(ALPHABET))(x)
    x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(MAX_SEQUENCE_LENGTH*len(ALPHABET))(x)

x = tf.reshape(x,(-1, MAX_SEQUENCE_LENGTH, len(ALPHABET)))

x = tf.keras.activations.softmax(x)

model = tf.keras.Model([input_layers[MZ],input_layers[INT]],x)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy())
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2354)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2354)]       0                                            
__________________________________________________________________________________________________
tf.__operators__.add (TFOpLambd (None, 2354)         0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2354)         0           tf.__operators__.add[0][0]   

## Training the Tensorflow Model

In [30]:
def split_dataset(dataset, fraction):
    split_value = int(len(dataset) * fraction)
    a = dataset.take(split_value)
    b = dataset.skip(split_value)
    return a, b

In [31]:
dataset = dataset.shuffle(buffer_size=int(10000 / BATCH_SIZE))

In [32]:
model.fit(dataset, epochs=10)

Epoch 1/10
21126/21126 [==============================] - 877s 41ms/step - loss: 6.0935
Epoch 2/10
  209/21126 [..............................] - ETA: 15:32 - loss: 5.4155

KeyboardInterrupt: 

## Evaluating the Tensorflow Model